LoanTap is an online platform committed to delivering customized loan products to millennials. They innovate in an otherwise dull loan segment, to deliver instant, flexible loans on consumer friendly terms to salaried professionals and businessmen.

The data science team at LoanTap is building an underwriting layer to determine the creditworthiness of MSMEs as well as individuals.

LoanTap deploys formal credit to salaried individuals and businesses 4 main financial instruments:

Personal Loan
EMI Free Loan
Personal Overdraft
Advance Salary Loan
This case study will focus on the underwriting process behind Personal Loan only

### Understanding the data

loan_amnt : The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

term : The number of payments on the loan. Values are in months and can be either 36 or 60.

int_rate : Interest Rate on the loan

installment : The monthly payment owed by the borrower if the loan originates.

grade : LoanTap assigned loan grade

sub_grade : LoanTap assigned loan subgrade

emp_title :The job title supplied by the Borrower when applying for the loan.*

emp_length : Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.

home_ownership : The home ownership status provided by the borrower during registration or obtained from the credit report.

annual_inc : The self-reported annual income provided by the borrower during registration.

verification_status : Indicates if income was verified by LoanTap, not verified, or if the income source was verified

issue_d : The month which the loan was funded

loan_status : Current status of the loan - Target Variable

purpose : A category provided by the borrower for the loan request.

title : The loan title provided by the borrower

dti : A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LoanTap loan, divided by the borrower’s self-reported monthly income.

earliest_cr_line :The month the borrower's earliest reported credit line was opened

open_acc : The number of open credit lines in the borrower's credit file.

pub_rec : Number of derogatory public records

revol_bal : Total credit revolving balance

revol_util : Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

total_acc : The total number of credit lines currently in the borrower's credit file

initial_list_status : The initial listing status of the loan. Possible values are – W, F

application_type : Indicates whether the loan is an individual application or a joint application with two co-borrowers

mort_acc : Number of mortgage accounts.

pub_rec_bankruptcies : Number of public record bankruptcies

Address: Address of the individual

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, precision_recall_curve, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE
import warnings

In [ ]:
df = pd.read_csv("logistic_regression.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
df.info() 

In [ ]:
for i in df.columns:
    print(i, '-->> ', df[i].unique(), '\n')

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

Dropping all null values

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

There are no duplicates in the dataset


## Univariate Analysis

In [ ]:
df.columns

In [ ]:
float_columns = df.select_dtypes(include=['float64']).columns
float_columns

In [ ]:
df_float = df[float_columns]
df_float

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_float.corr(method='spearman'),annot=True,cmap='viridis')
plt.show()

We noticed almost perfect correlation between "loan_amnt" the "installment" feature.

installment: The monthly payment owed by the borrower if the loan originates.
loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
So, we can drop either one of those columns.

In [ ]:
df.drop(columns=['installment'],axis=1,inplace=True)
df_float.drop(columns=['installment'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_float.corr(method='spearman'),annot=True,cmap='viridis')
plt.show()

In [ ]:
univariate_cols = df.select_dtypes('float64').columns.tolist()

In [ ]:
for i in univariate_cols:
#     plt.figure(figsize=(12,5))
    plt.title("Distribution of {}".format(i))
    sns.histplot(df[i]/df[i].max(), kde=True, bins=50)
    plt.show()

Most of the distribution is highly skewed which tells us that they might contain outliers
Almost all the continuous features have outliers present in the dataset. They have to be standarsised


In [ ]:
cat_vars = ['home_ownership', 'verification_status', 'loan_status', 'application_type', 'grade', 'sub_grade', 'term']
for i in cat_vars: 
    plt.figure(figsize=(10, 4))
    plt.title(f'Distribution of {i}')
    sns.countplot(data=df, x=i)
    plt.xticks(rotation = 45)
    plt.show()

All the application type is Individual

Most of the loan tenure is disbursed for 36 months

The grade of majority of people those who have took the loan is 'B' and have subgrade 'B3'.

So from that we can infer that people with grade 'B' and subgrade 'B3' are more likely to fully pay the loan.

Most of the people took loan for 36 months and full paid on time

Most of people have home ownership as mortgage and rent

Most of the people took loan for debt consolidations

All the application type is Individual
Most of the loan tenure is disbursed for 36 months
The grade of majority of people those who have took the loan is 'B' and have subgrade 'B3'.
So from that we can infer that people with grade 'B' and subgrade 'B3' are more likely to fully pay the loan.

### Outlier Treatment

In [ ]:
count = 0
plt.figure(figsize=(20,30))
for i in univariate_cols:
    count += 1
    plt.subplot(5,3,count)
    sns.boxplot(y= df[i])

In [ ]:
df.shape

In [ ]:
for col in univariate_cols:
    mean=df[col].mean()
    std=df[col].std()
    
    upper_limit=mean+3*std
    lower_limit=mean-3*std
    
    df=df[(df[col]<upper_limit) & (df[col]>lower_limit)]
    
df.shape

In [ ]:
plt.figure(figsize=(15,20))

plt.subplot(4,2,1)
sns.countplot(x='term',data=df,hue='loan_status')

plt.subplot(4,2,2)
sns.countplot(x='home_ownership',data=df,hue='loan_status')

plt.subplot(4,2,3)
sns.countplot(x='verification_status',data=df,hue='loan_status')

plt.subplot(4,2,4)
g=sns.countplot(x='purpose',data=df,hue='loan_status')
g.set_xticklabels(g.get_xticklabels(),rotation=90)

plt.show()

Most of the people took loan for 36 months and full paid on time

Most of people have home ownership as mortgage and rent

Most of the people took loan for debt consolidations

4. People with grades ‘A’ are more likely to fully pay their loan. (T/F)

In [ ]:
plt.figure(figsize=(15, 10))
warnings.filterwarnings("ignore")

plt.subplot(2, 2, 1)
grade = sorted(df.grade.unique().tolist())
sns.countplot(x='grade', data=df, hue='loan_status', order=grade)

plt.subplot(2, 2, 2)
sub_grade = sorted(df.sub_grade.unique().tolist())
g = sns.countplot(x='sub_grade', data=df, hue='loan_status', order=sub_grade)
g.set_xticklabels(g.get_xticklabels(), rotation=90)
plt.show()


The grade of majority of people those who have fully paid the loan is 'B' and have subgrade 'B3'.

So from that we can infer that people with grade 'B' and subgrade 'B3' are more likely to fully pay the loan.

5. Name the top 2 afforded job titles.

In [ ]:
plt.figure(figsize=(15,12))

plt.subplot(2,2,1)
order = ['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', 
          '6 years', '7 years', '8 years', '9 years', '10+ years',]
g=sns.countplot(x='emp_length',data=df,hue='loan_status',order=order)
g.set_xticklabels(g.get_xticklabels(),rotation=90)

plt.subplot(2,2,2)
plt.barh(df.emp_title.value_counts()[:30].index,df.emp_title.value_counts()[:30])
plt.title("The most 30 jobs who took a loan")
plt.tight_layout()
plt.show()

Manager and Teacher are the most afforded loan on titles

Person who employed for more than 10 years has successfully paid of the loan

In [ ]:
def f1(number):
    if number == 0.0:
        return 0
    else:
        return 1
    
def f2(number):
    if number == 0.0:
        return 0
    elif number >= 1.0:
        return 1
    else:
        return number

In [ ]:
df['pub_rec']=df.pub_rec.apply(f1)
df['mort_acc']=df.mort_acc.apply(f2)
df['pub_rec_bankruptcies']=df.pub_rec_bankruptcies.apply(f2)

In [ ]:
plt.figure(figsize=(12,30))

plt.subplot(6,2,1)
sns.countplot(x='pub_rec',data=df,hue='loan_status')

plt.subplot(6,2,2)
sns.countplot(x='initial_list_status',data=df,hue='loan_status')

plt.subplot(6,2,3)
sns.countplot(x='application_type',data=df,hue='loan_status')

plt.subplot(6,2,4)
sns.countplot(x='mort_acc',data=df,hue='loan_status')

plt.subplot(6,2,5)
sns.countplot(x='pub_rec_bankruptcies',data=df,hue='loan_status')

plt.show()

Most the loan disbursed to the people whose do not hold bankrupties record have successfully paid loan


## Feature Engineering

In [ ]:
for i in df.columns:
    print(i, '-->> ', df[i].unique(), '\n')

In [ ]:
# Converting term values to numerical val
term_values={' 36 months': 36, ' 60 months':60}
df['term'] = df.term.map(term_values)

# Mapping the target variable
df['loan_status'] = df.loan_status.map({'Fully Paid':0, 'Charged Off':1})

# Initial List Status
df['initial_list_status'].unique()
np.array(['w', 'f'], dtype=object)
list_status = {'w': 0, 'f': 1}
df['initial_list_status'] = df.initial_list_status.map(list_status)

# Let's fetch ZIP from address and then drop the remaining details -
df['zip_code'] = df.address.apply(lambda x: x[-5:]) 
df['zip_code'].value_counts(normalize=True)*100

for i in df.columns:
    print(i, '-->> ', df[i].unique(), '\n')

In [ ]:
df.head()

In [ ]:
df['issue_d'].head()

In [ ]:
df['issue_month'] = df['issue_d'].apply(lambda x : str(x).split('-')[0])
df['issue_year'] = df['issue_d'].apply(lambda x : str(x).split('-')[1])

In [ ]:
df = df.drop(columns=['issue_d'], axis=1)

In [ ]:
df[['earliest_cr_line']].head()

In [ ]:
df['earliest_cr_line_month'] = df['earliest_cr_line'].apply(lambda x : str(x).split('-')[0])
df['earliest_cr_line_year'] = df['earliest_cr_line'].apply(lambda x : str(x).split('-')[1])

In [ ]:
df = df.drop(columns=['earliest_cr_line'], axis=1)

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['address','zip_code','title',], axis=1)

In [ ]:
df.columns

In [ ]:
for col in df.columns:
  print(col, '->', df[col].nunique())

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df['term'] = label_encoder.fit_transform(df['term'])
df['grade'] = label_encoder.fit_transform(df['grade'])
df['sub_grade'] = label_encoder.fit_transform(df['sub_grade'])
df['emp_length'] = label_encoder.fit_transform(df['emp_length'])
df['home_ownership'] = label_encoder.fit_transform(df['home_ownership'])
df['verification_status'] = label_encoder.fit_transform(df['verification_status'])
df['loan_status'] = label_encoder.fit_transform(df['loan_status'])
df['purpose'] = label_encoder.fit_transform(df['purpose'])
df['pub_rec'] = label_encoder.fit_transform(df['pub_rec'])
df['initial_list_status'] = label_encoder.fit_transform(df['initial_list_status'])
df['application_type'] = label_encoder.fit_transform(df['application_type'])
df['mort_acc'] = label_encoder.fit_transform(df['mort_acc'])
df['pub_rec_bankruptcies'] = label_encoder.fit_transform(df['pub_rec_bankruptcies'])
df['open_acc'] = label_encoder.fit_transform(df['open_acc'])
df['issue_month'] = label_encoder.fit_transform(df['issue_month'])
df['issue_year'] = label_encoder.fit_transform(df['issue_year'])
df['earliest_cr_line_month'] = label_encoder.fit_transform(df['earliest_cr_line_month'])
df['earliest_cr_line_year'] = label_encoder.fit_transform(df['earliest_cr_line_year'])

In [ ]:
df.head()

In [ ]:
df1 = pd.DataFrame({'emp_title' : df['emp_title'], 'target' : df['loan_status']})
target_mean = df1.groupby(by=['emp_title'])['target'].mean()
df['emp_title'] = df1['emp_title'].map(target_mean)
df.head()

In [ ]:
X=df.drop('loan_status',axis=1)
y=df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,stratify=y,random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

MinMaxScaler -
For each value in a feature, MinMaxScaler subtracts the minimum value in the feature and then divides by the range. The range is the difference between the original maximum and original minimum.

MinMaxScaler preserves the shape of the original distribution. It doesn’t meaningfully change the information embedded in the original data.

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.score(X_test,y_test)

Accuracy of Logistic Regression Classifier on test set: 0.888

In [ ]:
x_sm = sm.add_constant(X_train)
sm_model = sm.OLS(y_train, x_sm)
result = sm_model.fit()
print(result.summary())

In [ ]:
dic = {}
for coef, col in zip(model.coef_[0], df.columns):
  dic[col] = abs(coef)
a = sorted(dic.items(), key = lambda x: (x[1], x[0]))
for i in a:
  print(i)

In [ ]:
cm = confusion_matrix(y_test,y_pred)
print(cm)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_).plot()

There is significant value for false negative and false positive. Which will hamper our prediction due to type-1 or type-2 error.


In [ ]:
print(classification_report(y_test,y_pred))

Precision score and recall score for full paid status is almost same indicates that model is doing decent job which correctly classified the both of the scenarios

Precision score for charged off status is more than recall score which is perfect

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

ROC Curve -
An ROC curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots two parameters:

True Positive Rate
False Positive Rate
True Positive Rate (TPR) is a synonym for recall and is therefore defined as follows:

TPR=(TP)/(TP+FN)
False Positive Rate (FPR) is defined as follows:

FPR=(FP)/(FP+TN)
An ROC curve plots TPR vs. FPR at different classification thresholds. Lowering the classification threshold classifies more items as positive, thus increasing both False Positives and True Positives. The following figure shows a typical ROC curve.

AUC (Area under the ROC Curve) -
AUC stands for "Area under the ROC Curve." That is, AUC measures the entire two-dimensional area underneath the entire ROC curve (think integral calculus) from (0,0) to (1,1).

AUC provides an aggregate measure of performance across all possible classification thresholds. One way of interpreting AUC is as the probability that the model ranks a random positive example more highly than a random negative example. For example, given the following examples, which are arranged from left to right in ascending order of logistic regression predictions.

In [ ]:
prob = (model.predict_proba(X_test))[:,1]
fpr, tpr, thr = roc_curve(y_test, prob)
logit_roc_auc = roc_auc_score(y_test,model.predict(X_test))

plt.plot(fpr,tpr)
plt.plot(fpr,fpr,'--',color='red' )
plt.plot(fpr,tpr,label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.title('ROC curve')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

ROC-AUC curve is grossing the area which indicates that model is performing well.
There is still room for some model improvement

By collecting more data, using a more complex model, or tuning the hyperparameters, it is possible to improve the model's performance.

In [ ]:
roc_auc_score(y_test,prob)

In [ ]:
def precission_recall_curve_plot(y_test,pred_proba_c1):
    precisions, recalls, thresholds = precision_recall_curve(y_test,pred_proba_c1)
    
    threshold_boundary = thresholds.shape[0]
    #plot precision
    plt.plot(thresholds,precisions[0:threshold_boundary],linestyle='--',label='precision')
    #plot recall
    plt.plot(thresholds,recalls[0:threshold_boundary],label='recalls')
    
    start,end=plt.xlim()
    plt.xticks(np.round(np.arange(start,end,0.1),2))
    
    plt.xlabel('Threshold Value')
    plt.ylabel('Precision and Recall Value')
    plt.legend()
    plt.grid()
    plt.show()

precission_recall_curve_plot(y_test,model.predict_proba(X_test)[:,1])

Multicollinearity check using Variance Inflation Factor (VIF) -
Multicollinearity occurs when two or more independent variables are highly correlated with one another in a regression model. Multicollinearity can be a problem in a regression model because we would not be able to distinguish between the individual effects of the independent variables on the dependent variable.

Multicollinearity can be detected via various methods. One such method is Variance Inflation Factor aka VIF. In VIF method, we pick each independent feature and regress it against all of the other independent features. VIF score of an independent variable represents how well the variable is explained by other independent variables.

In [ ]:
def calc_vif(X):
    # Calculating the VIF
    vif=pd.DataFrame()
    vif['Feature']=X.columns
    vif['VIF']=[variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
    vif['VIF']=round(vif['VIF'],2)
    vif=vif.sort_values(by='VIF',ascending=False)
    return vif

calc_vif(X)[:5]

In [ ]:
X.drop(columns=['int_rate'],axis=1,inplace=True)
calc_vif(X)[:5]

In [ ]:
X.drop(columns=['application_type'],axis=1,inplace=True)
calc_vif(X)[:5]

In [ ]:
X.drop(columns=['sub_grade'],axis=1,inplace=True)
calc_vif(X)[:5]

In [ ]:
X.drop(columns=['earliest_cr_line_year'],axis=1,inplace=True)
calc_vif(X)[:5]

In [ ]:
X.drop(columns=['total_acc'],axis=1,inplace=True)
calc_vif(X)[:5]

In [ ]:
X=scaler.fit_transform(X)

kfold=KFold(n_splits=5)
accuracy=np.mean(cross_val_score(model,X,y,cv=kfold,scoring='accuracy',n_jobs=-1))
print("Cross Validation accuracy : {:.3f}".format(accuracy))

In [ ]:
sm=SMOTE(random_state=42)
X_train_res,y_train_res=sm.fit_resample(X_train,y_train.ravel())

In [ ]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))

In [ ]:
lr1 = LogisticRegression(max_iter=1000)
lr1.fit(X_train_res, y_train_res)
predictions = lr1.predict(X_test)
  
# Classification Report
print(classification_report(y_test, predictions))

In [ ]:
precission_recall_curve_plot(y_test, lr1.predict_proba(X_test)[:,1])

In [ ]:
df = pd.read_csv('logistic_regression.csv')

1. What percentage of customers have fully paid their Loan Amount?

In [ ]:
df ['loan_status'].value_counts(normalize=True)*100

80%

2. Comment about the correlation between Loan Amount and Installment features.


The spearman correlation coefficient between Loan Amount and Installmen is very high (i.e. 0.97)

3. The majority of people have home ownership as _______.

In [ ]:
df ['home_ownership'].value_counts(normalize=True)*100

In [ ]:
df['home_ownership'].value_counts()

Mortgage

4. People with grades ‘A’ are more likely to fully pay their loan. (T/F)

True.

Out of all people with grade 'A', 93% got their loan approved.

5. Name the top 2 afforded job titles.

Teacher & Manager

6. Thinking from a bank's perspective, which metric should our primary focus be on..
ROC AUC
Precision
Recall
F1 Score

It should be on f1 score.  as we need to give importance to both precision and recall. We don't want to miss potential customers and at the same time we also don't want to give loan to defaulters

7. How does the gap in precision and recall affect the bank?


Recall score: 0.94 and Precision score: 0.85. Which tells us that there are more false positives than the false negatives.

From Confusion Matrix it can be seen that FP = 10% of total cases & FN = 0.9% of Total Cases

If Recall value is low (i.e. FN are high), it means Bank is loosing in opportunity cost.

If Precision value is low (i.e. FP are high), it means Bank's NPA (defaulters) may increase.

8. Which were the features that heavily affected the outcome?

Using RFE we were able to identify top_20 features which has high impact on Outcome. This include:

int_rate: Interest Rate

sub_grade: loan subgrade

term : number of payments on the loan

home_ownership

purpose

application_type

pincode (from address)

emp_title: job title supplied by the Borrower

9. Will the results be affected by geographical location? (Yes/No)

pincode (derived from address) has significant impact on the outcome.

How can we make sure that our model can detect real defaulters and there are less false positives? This is important as we can lose out on an opportunity to finance more individuals and earn interest on it.

Answer - Since data is imbalances by making the data balance we can try to avoid false positives. For evaluation metrics, we should be focusing on the macro average f1-score because we don't want to make false positive prediction and at the same we want to detect the defualers.
Since NPA (non-performing asset) is a real problem in this industry, it’s important we play safe and shouldn’t disburse loans to anyone

Answer - Below are the most features and their importance while making the prediction. So these variables can help the managers to identify which are customers who are more likely to pay the loan amount fully,

## Actional Insights and Recommendations


80% of the customers have paid the loan fully.

20% of the customers are the defaulters.

The organization can the trained model to make prediction for whether a person will likely to pay the loan amount or he will be a defaulter.

Cross Validation accuracy and testing accuracy is almost same which infers model is performing the decent job. We can trust this model for unseen data

By collecting more data, using a more complex model, or tuning the hyperparameters, it is possible to improve the model's performance.

ROC AUC curve area of 0.73, the model is correctly classifying about 73% of the instances. This is a good performance, but there is still room for improvement.

The precision-recall curve allows us to see how the precision and recall trade-off as we vary the threshold. A higher threshold will result in higher precision, but lower recall, and vice versa. The ideal point on the curve is the one that best meets the needs of the specific application.

After balancing the dataset, there is significant change observed in the precion and recall score for both of the classes.

Accuracy of Logistic Regression Classifier on test set: 0.888 which is decent and not by chance.